In [1]:
import timeit
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tables as tb

In [2]:
from torch.utils.data import Dataset
from MF_RP_mat_h5support import H5_NODE_NAME
m = nn.Conv2d(16, 33, 3, padding=0)

input = torch.randn(20,16,50,100)
_input = input.numpy()
out = m(input)

In [3]:
pool = nn.MaxPool2d(2, stride=2, return_indices=True)
unpool = nn.MaxUnpool2d(2, stride=2)
input = torch.tensor([[[[ 1.,  2,  3,  4],
                            [ 5,  6,  7,  8],
                            [ 9, 10, 11, 12],
                            [13, 14, 15, 16]]]])
output, indices = pool(input)
a=pool(input)

In [4]:
a = torch.randn((3, 10))
b = torch.randn((3,10))
c = a -b
d= c**2
dT= d.T

In [5]:
#***STUDENT T DISTRIBUTION TEST***
#q_ij = 1/(1+dist(z_i, u_j)^2), then normalize it.
centroids = np.array(
    [[1,1,1,1],
    [3,3,3,3],
    [6,6,6,6]]
) #u, 3 classes, each class centroid is 4D vector
emb1 = np.array([[13,14,15,16],])
emb2 = np.array([[17,18,19,20],])
embs = np.array([emb1, emb2])#z, 2 samples, each sample embedded to 4D vector

z_minus_u = embs - centroids # diff between each z and each centroids
print('centroids(u):')
print(centroids)
print('---')
print('embs(z):')
print(embs)
print('---')
print('z_minus_u:')
print(z_minus_u)
z_minus_u_square = np.square(z_minus_u)  # i.e., distance square between each z and each centroids
dist_square_sum = np.sum(z_minus_u_square, axis=2) # each element: square of euclid distance
print('---')
print('z_minus_u_square:')
print(z_minus_u_square)
print('---')
print('dist_square_sum:')
print(dist_square_sum)
alpha = 1.
qij = 1. / (1. + dist_square_sum / 1.) # each element: the probability of sample i belong to j
print('---')
print('qij:')
print(qij)
qij_sum = np.sum(qij, axis=1) # sum probability to all 3 centroid
print('---')
print('qij_sum:')
print(qij_sum)
qij_t = np.transpose(qij)
print('---')
print('qij_t:')
print(qij_t)
qij_normalize = qij_t / qij_sum # each probability/sum probability
print('---')
print('qij_normalize:')
print(qij_normalize)
print('---')
print('qij_normalize_t:')
print(qij_normalize.T) # transpose back to denote the probability of i belong to j

centroids(u):
[[1 1 1 1]
 [3 3 3 3]
 [6 6 6 6]]
---
embs(z):
[[[13 14 15 16]]

 [[17 18 19 20]]]
---
z_minus_u:
[[[12 13 14 15]
  [10 11 12 13]
  [ 7  8  9 10]]

 [[16 17 18 19]
  [14 15 16 17]
  [11 12 13 14]]]
---
z_minus_u_square:
[[[144 169 196 225]
  [100 121 144 169]
  [ 49  64  81 100]]

 [[256 289 324 361]
  [196 225 256 289]
  [121 144 169 196]]]
---
dist_square_sum:
[[ 734  534  294]
 [1230  966  630]]
---
qij:
[[0.00136054 0.00186916 0.00338983]
 [0.00081235 0.00103413 0.00158479]]
---
qij_sum:
[0.00661953 0.00343126]
---
qij_t:
[[0.00136054 0.00081235]
 [0.00186916 0.00103413]
 [0.00338983 0.00158479]]
---
qij_normalize:
[[0.20553476 0.2367491 ]
 [0.28237018 0.3013838 ]
 [0.51209507 0.4618671 ]]
---
qij_normalize_t:
[[0.20553476 0.28237018 0.51209507]
 [0.2367491  0.3013838  0.4618671 ]]


In [27]:
#How to calculate the mean and std of my own dataset?
#https://discuss.pytorch.org/t/about-normalization-using-pre-trained-vgg16-networks/23560/23
class MyDataset(Dataset):
    def __init__(self):
        self.data = torch.randn(1000, 3, 224, 224)

    def __getitem__(self, index):
        x = self.data[index]
        return x

    def __len__(self):
        return len(self.data)

def main():
    device = torch.device("cpu")
    dataset = MyDataset()

    start = timeit.time.perf_counter()
    data = dataset.data.to(device)
    print("Mean:", torch.mean(data, dim=(0, 2, 3))) # get mean on each channel for all samples
    print("Std:", torch.std(data, dim=(0, 2, 3)))
    print("Elapsed time: %.3f seconds" % (timeit.time.perf_counter() - start))
    print()

    start = timeit.time.perf_counter()
    mean = 0.
    for data in dataset:
        data = data.to(device)
        mean += torch.mean(data, dim=(1, 2))
    mean /= len(dataset)
    print("Mean:", mean)

    temp = 0.
    nb_samples = 0.
    for data in dataset:
        data = data.to(device)
        temp += ((data.view(3, -1) - mean.unsqueeze(1)) ** 2).sum(dim=1)
        nb_samples += np.prod(data.size()[1:])
    std = torch.sqrt(temp/nb_samples)
    print("Std:", std)
    print("Elapsed time: %.3f seconds" % (timeit.time.perf_counter() - start))
main()

Mean: tensor([ 6.3159e-05,  1.4881e-04, -6.2440e-05])
Std: tensor([0.9999, 0.9999, 0.9999])
Elapsed time: 0.760 seconds

Mean: tensor([ 6.3159e-05,  1.4881e-04, -6.2440e-05])
Std: tensor([0.9999, 0.9999, 0.9999])
Elapsed time: 0.129 seconds


In [13]:
data_type = 'train'
RP_mats_h5array = tb.open_file(f'./data/SHL_features/RP_mats_{data_type}.h5', mode='r').get_node('/' + H5_NODE_NAME)
RP_mats_h5array.shape
e=RP_mats_h5array[:]

In [26]:
x=torch.arange(24, dtype=torch.float).view(2,3,4)
print(x)
x_mean0=torch.mean(x,dim=(0,1),keepdim=False)
x_mean1=torch.mean(x,dim=0,keepdim=False)
x_mean2=torch.mean(x,dim=1,keepdim=False)
print(x_mean0)
print(x_mean1)
print(x_mean2)

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])
tensor([10., 11., 12., 13.])
tensor([[ 6.,  7.,  8.,  9.],
        [10., 11., 12., 13.],
        [14., 15., 16., 17.]])
tensor([[ 4.,  5.,  6.,  7.],
        [16., 17., 18., 19.]])
